In [1]:
import torch, detectron2
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Tue_May__3_18:49:52_PDT_2022
Cuda compilation tools, release 11.7, V11.7.64
Build cuda_11.7.r11.7/compiler.31294372_0


/home/djjin/anaconda3/envs/conda_visual_HPE/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


torch:  1.13 ; cuda:  cu117
detectron2: 0.6


In [1]:
from detectron2.utils.logger import setup_logger
setup_logger()

import os, sys
home_dir = os.path.abspath(os.getcwd()+"/../")
sys.path.append(home_dir)
home_dir = os.path.abspath(os.getcwd())
sys.path.append(home_dir)
print(home_dir)

from utils.arguments import load_vcoco_opt_command
from utils.distributed import init_distributed
from datasets.build import build_eval_dataloader

/home/djjin/anaconda3/envs/conda_visual_HPE/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Invalid MIT-MAGIC-COOKIE-1 key

/home/djjin/Mygit/X-Decoder/notebooks


datasets/ADE20K_2021_17_01/images_detectron2/training
datasets/ADE20K_2021_17_01/images_detectron2/validation


In [9]:
opt, cmdline_args = load_vcoco_opt_command(home_dir)
opt = init_distributed(opt)

In [32]:
def get_config_from_name(cfg, dataset_name):
    # adjust config according to dataset
    print(dataset_name)
    if 'refcoco' in dataset_name:
        cfg.update(cfg['REF'])
        return cfg
    elif 'coco' in dataset_name:
        if 'COCO' in cfg.keys():
            cfg.update(cfg['COCO'])
        return cfg
    elif 'ade' in dataset_name:
        if 'ADE20K' in cfg.keys():
            cfg.update(cfg['ADE20K'])
        return cfg
    elif 'imagenet' in dataset_name:
        if 'IMAGENET' in cfg.keys():
            cfg.update(cfg['IMAGENET'])
        return cfg
    elif 'vlp' in dataset_name:
        cfg.update(cfg['VLP'])
        return cfg
    elif 'sun' in dataset_name:
        cfg.update(cfg['SUN'])
        return cfg
    elif 'scan' in dataset_name:
        cfg.update(cfg['SCAN'])
        return cfg
    elif 'cityscape' in dataset_name:
        cfg.update(cfg['CITY'])
        return cfg
    elif 'bdd' in dataset_name:
        cfg.update(cfg['BDD'])
        return cfg
    elif 'vcoco' in dataset_name:
        cfg.update(cfg['VCOCO'])
    else:
        assert False, "dataset not support."

In [33]:
from detectron2.data.common import DatasetFromList, MapDataset
from detectron2.data.dataset_mapper import DatasetMapper
from detectron2.data.samplers import InferenceSampler, TrainingSampler
from xdecoder.utils import configurable
from datasets.build import _train_loader_from_config
from detectron2.data.build import build_batch_data_loader
from datasets.build import JointLoader

@configurable(from_config=_train_loader_from_config)
def build_detection_train_loader(
    dataset, *, mapper, sampler=None, total_batch_size, aspect_ratio_grouping=True, num_workers=0
):
    """
    Build a dataloader for object detection with some default features.
    This interface is experimental.

    Args:
        dataset (list or torch.utils.data.Dataset): a list of dataset dicts,
            or a map-style pytorch dataset. They can be obtained by using
            :func:`DatasetCatalog.get` or :func:`get_detection_dataset_dicts`.
        mapper (callable): a callable which takes a sample (dict) from dataset and
            returns the format to be consumed by the model.
            When using cfg, the default choice is ``DatasetMapper(cfg, is_train=True)``.
        sampler (torch.utils.data.sampler.Sampler or None): a sampler that
            produces indices to be applied on ``dataset``.
            Default to :class:`TrainingSampler`, which coordinates a random shuffle
            sequence across all workers.
        total_batch_size (int): total batch size across all workers. Batching
            simply puts data into a list.
        aspect_ratio_grouping (bool): whether to group images with similar
            aspect ratio for efficiency. When enabled, it requires each
            element in dataset be a dict with keys "width" and "height".
        num_workers (int): number of parallel data loading workers

    Returns:
        torch.utils.data.DataLoader: a dataloader. Each output from it is a
            ``list[mapped_element]`` of length ``total_batch_size / num_workers``,
            where ``mapped_element`` is produced by the ``mapper``.
    """
    if isinstance(dataset, list):
        dataset = DatasetFromList(dataset, copy=False)
    if mapper is not None:
        dataset = MapDataset(dataset, mapper)
    if sampler is None:
        sampler = TrainingSampler(len(dataset))
    assert isinstance(sampler, torch.utils.data.sampler.Sampler)
    return build_batch_data_loader(
        dataset,
        sampler,
        total_batch_size,
        aspect_ratio_grouping=aspect_ratio_grouping,
        num_workers=num_workers,
    )

In [34]:
from datasets.dataset_mappers import MaskFormerSemanticDatasetMapper
def build_train_dataloader(cfg, ):
    dataset_names = cfg['DATASETS']['TRAIN']
    loaders = {}
    for dataset_name in dataset_names:
        cfg = get_config_from_name(cfg, dataset_name)
        mapper_name = cfg['INPUT']['DATASET_MAPPER_NAME']
        # Semantic segmentation dataset mapper
        print(mapper_name)
        if mapper_name == "vcoco":
            mapper = MaskFormerSemanticDatasetMapper(cfg, True)
            loaders['coco'] = build_detection_train_loader(cfg, dataset_name=dataset_name, mapper=mapper)

    if len(loaders) == 1 and not cfg['LOADER'].get('JOINT', False):
        return list(loaders.values())[0]
    else:
        return JointLoader(loaders, key_dataset=cfg['LOADER'].get('KEY_DATASET', 'coco'))

In [36]:
dataloaders = build_train_dataloader(opt)
dataset_names = opt['DATASETS']['TEST']

vcoco_train
coco_panoptic_lsj


KeyError: 'LOADER'

In [6]:
dataloaders = build_eval_dataloader(opt)
dataset_names = opt['DATASETS']['TEST']

[07/12 15:24:14 d2.data.common]: Serializing 4946 elements to byte tensors and concatenating them all ...
[07/12 15:24:14 d2.data.common]: Serialized dataset takes 3.27 MiB


In [7]:
import cv2
import random
from datasets.utils.vcoco_utils import valid_obj_ids, get_coco_instance_ID_to_name
from matplotlib import pyplot as plt


def random_color():
    rdn = random.randint(1, 1000)
    b = int(rdn * 997) % 255
    g = int(rdn * 4447) % 255
    r = int(rdn * 6563) % 255
    return b, g, r


for dataloader, dataset_name in zip(dataloaders, dataset_names):
    with torch.no_grad():
        for idx, batch in enumerate(dataloader):
            for test in batch:
                print(test.keys())
                # target_img = test["image"]
                # target = test["instances"]
                # img = target_img.permute(1, 2, 0).detach().cpu().numpy()
                # hh, ww = img.shape[0], img.shape[1]
                # # img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                # # img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

                # # obj_labels = []
                # # labels = []
                # # verb_names = []
                # # ob_boxes = []
                # # sub_boxes = []

                # # obj_boxes = target['boxes']
                # # for test in range(obj_boxes.shape[0]):
                # #     cx, cy, w, h = obj_boxes[test]
                # #     color = random_color()
                # #     cx, cy, w, h = cx * ww, cy * hh, w * ww, h * hh
                # #     n_box = list(map(int, [cx - 0.5 * w, cy - 0.5 * h, cx + 0.5 * w, cy + 0.5 * h]))
                # #     x1, y1, x2, y2 = n_box
                # #     ob_boxes.append(n_box)

                # # for idx in target['labels']:
                # #     label_name = get_coco_instance_ID_to_name(valid_obj_ids[idx])
                # #     labels.append(label_name)

                # # for label, box in zip(labels, ob_boxes):
                # #     color = random_color()
                # #     x1, y1, x2, y2 = box
                # #     cv2.rectangle(img, (x1, y1), (x2, y2), color, 2)

                # #     x = (x1 + x2) / 2
                # #     y = (y1 + y2) / 2

                # #     plt.scatter(x, y, 30, color=[i/255 for i in color])
                # #     plt.text(
                # #         x + 5,
                # #         y + 5,
                # #         label,
                # #         fontsize=10,
                # #         bbox=dict(facecolor=[i/255 for i in color], alpha=0.5),
                # #         )

                # plt.imshow(img)
                # plt.show()
                

dict_keys(['id', 'file_name', 'hoi_annotation', 'annotations', 'image', 'instances'])
dict_keys(['id', 'file_name', 'hoi_annotation', 'annotations', 'image', 'instances'])
dict_keys(['id', 'file_name', 'hoi_annotation', 'annotations', 'image', 'instances'])
dict_keys(['id', 'file_name', 'hoi_annotation', 'annotations', 'image', 'instances'])
dict_keys(['id', 'file_name', 'hoi_annotation', 'annotations', 'image', 'instances'])
dict_keys(['id', 'file_name', 'hoi_annotation', 'annotations', 'image', 'instances'])
dict_keys(['id', 'file_name', 'hoi_annotation', 'annotations', 'image', 'instances'])
dict_keys(['id', 'file_name', 'hoi_annotation', 'annotations', 'image', 'instances'])
dict_keys(['id', 'file_name', 'hoi_annotation', 'annotations', 'image', 'instances'])
dict_keys(['id', 'file_name', 'hoi_annotation', 'annotations', 'image', 'instances'])
dict_keys(['id', 'file_name', 'hoi_annotation', 'annotations', 'image', 'instances'])
dict_keys(['id', 'file_name', 'hoi_annotation', 'annot

KeyboardInterrupt: 